In [1]:
#Import innitial dependencies
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Read the csv file into a pandas DataFrame
dataset = pd.read_csv('../data/ModelData_2.csv')
dataset.head()
#dataset.tail()

,Servings/s,Calories/s,Fat/s,Sat_Fat/s,Trans_Fat/s,Mono_Fat/s,Poly_Fat/s,Carbs/s,Fiber/s,Sugars/s,...,Vit_D/s,Vit_E/s,Vit_K/s,Water/s,CarbScore,CholoScore,TotalFatScore,SugarScore,HealthyGradScore,HealthyScore
0,1,100.783428,3.706567,0.478814,0.001744,1.190110,1.662028,12.935162,2.351610,0.802769,...,0.004374,0.598908,6.954946,58.288488,1,1,1,1,4,1
1,1,93.527653,3.372725,1.217548,0.058185,1.620999,0.330805,14.524590,4.714058,6.700123,...,1.065000,1.597009,50.059982,176.442965,1,1,1,1,4,1
2,1,101.527163,3.950959,0.675617,0.000000,2.540638,0.436856,14.242660,2.691121,3.243868,...,0.281250,1.147884,27.677754,119.293643,1,1,1,1,4,1
3,1,81.389119,3.745693,0.558064,0.000000,2.499658,0.501540,10.870846,4.103485,5.607749,...,0.000000,2.261851,62.910781,214.230832,1,1,1,1,4,1
4,1,76.274230,3.713546,0.424643,0.000000,1.780703,1.329019,9.893677,3.403681,5.222974,...,0.000000,1.356905,47.798536,137.342582,1,1,1,1,4,1


In [3]:
# Assign X (data) and y (target) - And review 
X = dataset.drop("HealthyScore", axis=1)
y = dataset["HealthyScore"]
print(X.shape, y.shape)

(1849, 37) (1849,)


In [4]:
# Split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [5]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
y_train_categorical

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [15]:
loaded_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model_loss, model_accuracy = loaded_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

463/463 - 0s - loss: 4.5958e-07 - accuracy: 1.0000
Normal Neural Network - Loss: 4.5957959721692324e-07, Accuracy: 1.0


In [25]:
encoded_predictions = loaded_model.predict_classes(X_test_scaled[:15])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [26]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:15])}")

Predicted classes: [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
Actual Labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
